# Recommendation System

Link de inputs:



Link de output: 


In [1]:
import pandas as pd
import numpy as np

import os
pathout = './data/'
if not os.path.exists(pathout):
    os.mkdir(pathout) 

### Descarga Google Sheets

In [2]:
from __future__ import print_function
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import json

#### 1. Traspasos Último mes

In [3]:
# Set the scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
# creds = Credentials.from_authorized_user_file('creds.json', scope)
# Authenticate with Google Sheets


client = gspread.authorize(creds)
SPREADSHEET_ID = 'xxxxxxxxx'
RANGE_NAME = "Traspasos!A1:Y"
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                             range=RANGE_NAME).execute()

values = result.get('values', [])

df = pd.DataFrame(values[1:], columns=values[0])

In [4]:
df.head(2)

,Año de Fecha,Trimestre de Fecha,Mes de Fecha,Vehículo ID Registro,Vehículo Marca,Vehículo Línea,Vehículo Modelo,Vehículo Color,Vehículo Carrocería,Segmento consolidado,...,Ciudad Probable Circulación,Departamento Registro,Ciudad Registro,Edad compra,Propietario Tipo Persona,Propietario Género,Estrato,,Year Group,MMA
0,2023,T2,junio,120,RENAULT,R9,1995,ROJO BORDEAUX PERLADO,SEDAN,AUTOMOVIL,...,BOGOTA,CUNDINAMARCA,COTA,Entre 31 y 40 años,PERSONA NATURAL,Masculino,3,1,(2010-2011),RENAULT R9 (2010-2011)
1,2023,T2,junio,1025,JEEP,GRAND CHEROKEE,2018,GRIS GRANITO,WAGON,UTILITARIO,...,BOGOTA,BOGOTA D.C.,BOGOTA,Entre 41 y 50 años,PERSONA NATURAL,Masculino,5,1,(2017-2018),JEEP GRAND CHEROKEE (2017-2018)


#### 2. Stock por MMA

In [5]:

# Set the scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope)
# creds = Credentials.from_authorized_user_file('creds.json', scope)
# Authenticate with Google Sheets


client = gspread.authorize(creds)
SPREADSHEET_ID = 'xxxxxxxxx'
RANGE_NAME = "Market Discount v2!A2:X"
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                             range=RANGE_NAME).execute()

values = result.get('values', [])

stock = pd.DataFrame(values[1:], columns=values[0])

stock = stock.drop('MMAT', axis=1)


In [6]:
list(stock)

['sku',
 'aod_make',
 'model',
 'year',
 'Year Group',
 'transmision',
 'Stock por SKU',
 'Stock por MMAT',
 'MD instant offer',
 'aod_market_price',
 'MMA',
 'Stock por MMA',
 'Opps L30D ',
 'Dias publicado',
 'D2S',
 'D2P',
 'Registros por SKU',
 'Inspecciones por SKU',
 'Dias publicado',
 'Oportunidades L30D',
 'Reservado L15D',
 'Ventas L60D',
 'MD base']

#### 3. Matriculas 2012

In [7]:
Matriculas = pd.read_csv(pathout + 'Matriculas desde 2012 - Jun2023.csv', encoding='utf-16',
                         delimiter='\t',
                         on_bad_lines='skip')


#Grupos de año
def year_group(row):

    if row['Año de Fecha'] < 2012:
        return "(2010-2011)"
    elif row['Año de Fecha'] < 2014:
        return "(2012-2013)"
    elif row['Año de Fecha'] < 2017:
        return "(2014-2016)"
    elif row['Año de Fecha'] < 2019:
        return "(2017-2018)"
    elif row['Año de Fecha'] < 2021:
        return "(2019-2020)"
    else:
        return "(2021-2023)"

# Aplicar la lógica a cada fila del DataFrame y crear una nueva columna llamada 'Resultado'
Matriculas['year group'] = Matriculas.apply(year_group, axis=1)

Matriculas['MMA'] = Matriculas['Vehículo Marca'].astype(str) + " " + Matriculas['Vehículo Línea'].astype(str) + " " + Matriculas['year group'].astype(str)


In [8]:
Matriculas_por_mma = Matriculas.groupby(['MMA'])['Año de Fecha'].count().reset_index()
Matriculas_por_mma.rename(columns={'Año de Fecha':'Matriculas'}, inplace=True)

Matriculas_por_mma.loc[Matriculas_por_mma['MMA'] == 'CHEVROLET SAIL (2014-2016)']

Matriculas_por_mma.sample(5)

,MMA,Matriculas
461,FIAT PUNTO (2014-2016),9
930,MERCEDES BENZ CLS 350 (2019-2020),1
1274,TOYOTA PRADO (2012-2013),10
29,AUDI A5 (2019-2020),8
1110,RAM 1000 (2021-2023),2


##### Limpieza

In [9]:
# Función para realizar el reemplazo en cada fila de la columna 'frase'
def reemplazar_palabras(frase):
    frase = frase.replace('CX-30', 'CX30')
    frase = frase.replace('CLIO III', 'CLIO')
    frase = frase.replace('CLIO II', 'CLIO')
    frase = frase.replace('SPORTAGE', 'NEW SPORTAGE')
    frase = frase.replace('ACCENT I25', 'ACCENT')

    
    return frase

# Aplicar la función a la columna 'frase'
stock['MMA'] = stock['MMA'].apply(reemplazar_palabras)




In [10]:
# list(stock)

In [11]:

column_names =['sku',
 'aod_make',
 'model',
 'year',
 'Year Group',
 'transmision',
 'Stock por SKU',
 'Stock por MMA',
 'MD instant offer',
 'aod_market_price',
 'MMA']

for column in column_names:
    stock[column] = stock[column].replace('#N/A', np.nan)
    stock[column] = stock[column].replace('None', np.nan)

stock = stock.replace('', pd.NA)
stock = stock.dropna(subset=['D2S', 'D2P', 'MD instant offer'])

columns = ['D2S', 'D2P', 'MD instant offer' , 'Registros por SKU', 'Inspecciones por SKU']

for column in columns:
    stock[column] = stock[column].str.replace(',', '.')
    stock[column] = stock[column].astype(float)


stock['Stock por MMA'] = stock['Stock por MMA'].astype(float)
stock['aod_market_price'] = pd.to_numeric(stock['aod_market_price'], errors='coerce').astype(float)




In [12]:
#fromkeys

# stock.loc[stock['MMA'] == 'SUZUKI SWIFT (2021-2023)']




In [13]:
avgmd = stock.groupby(['MMA'])['MD instant offer'].mean().reset_index()
avgmd.rename(columns={'MD instant offer':'AVG MD'}, inplace=True)


sdmd = stock.groupby(['MMA'])['MD instant offer'].std().reset_index()
sdmd.rename(columns={'MD instant offer':'SD MD'}, inplace=True)

##
avgd2s = stock.groupby(['MMA'])['D2S'].mean().reset_index()
avgd2s.rename(columns={'D2S':'AVG D2S'}, inplace=True)

avgd2p = stock.groupby(['MMA'])['D2P'].mean().reset_index()
avgd2p.rename(columns={'D2P':'AVG D2P'}, inplace=True)

avgregisters = stock.groupby(['MMA'])['Registros por SKU'].mean().reset_index()
avgregisters.rename(columns={'Registros por SKU':'AVG Registers'}, inplace=True)

avginspec = stock.groupby(['MMA'])['Inspecciones por SKU'].mean().reset_index()
avginspec.rename(columns={'Inspecciones por SKU':'AVG Inspections'}, inplace=True)

###
stock = stock.merge(avgmd, how='left', on=['MMA']).fillna(0)
stock = stock.merge(sdmd, how='left', on=['MMA']).fillna(0)
stock = stock.merge(avgd2s, how='left', on=['MMA']).fillna(0)
stock = stock.merge(avgregisters, how='left', on=['MMA']).fillna(0)
stock = stock.merge(avginspec, how='left', on=['MMA']).fillna(0)
stock = stock.merge(avgd2p, how='left', on=['MMA']).fillna(0)

stock = stock.drop_duplicates(subset=['MMA'])

stock.head(2)

stock_sorted = stock.sort_values('Stock por MMA', ascending=False)
stock_sorted['porcentaje'] = stock_sorted['Stock por MMA']/sum(stock_sorted['Stock por MMA'])
stock_sorted = stock_sorted[['MMA', 'porcentaje','Stock por MMA' ,'AVG MD' , 'SD MD' , 'aod_market_price', 'AVG D2S' , 'AVG D2P',
                             'AVG Registers', 'AVG Inspections', 'Dias publicado',	'Oportunidades L30D',	'Reservado L15D',	'Ventas L60D' ]]


#Crear la columna D2SC (D2S comercial):
stock_sorted['AVG D2S Com'] = stock_sorted['AVG D2S'] - stock_sorted['AVG D2P']


#Crear la columna MD$:
stock_sorted['MD$'] = stock_sorted['AVG MD'] * stock_sorted['aod_market_price']




stock_sorted.head(2)


,MMA,porcentaje,Stock por MMA,AVG MD,SD MD,aod_market_price,AVG D2S,AVG D2P,AVG Registers,AVG Inspections,Dias publicado,Dias publicado,Oportunidades L30D,Reservado L15D,Ventas L60D,AVG D2S Com,MD$
899,SUZUKI SWIFT (2021-2023),0.029817,13.0,0.212071,0.065651,0.0,81.323694,18.293082,4.928571,0.0,0,139,88,8,4,63.030612,0.000000e+00
3267,MAZDA 3 (2017-2018),0.020642,9.0,0.158833,0.026832,70958152.0,95.751917,20.481084,6.375000,0.0,0,54,49,2,2,75.270833,1.127052e+07


In [14]:
stock_sorted.loc[stock_sorted['MMA'] == 'SUZUKI SWIFT (2021-2023)']


,MMA,porcentaje,Stock por MMA,AVG MD,SD MD,aod_market_price,AVG D2S,AVG D2P,AVG Registers,AVG Inspections,Dias publicado,Dias publicado,Oportunidades L30D,Reservado L15D,Ventas L60D,AVG D2S Com,MD$
899,SUZUKI SWIFT (2021-2023),0.029817,13.0,0.212071,0.065651,0.0,81.323694,18.293082,4.928571,0.0,0,139,88,8,4,63.030612,0.0


In [15]:
agrupaciones = ['Vehículo Marca' , 'Vehículo Línea', 'Vehículo Modelo']
agrupaciones2 = ['MMA']

#Eliminar 2010 - 2012
df = df[~df['Year Group'].isin(['(2010-2011)', '(2011-2012)'])]




df_grouped = df.groupby(agrupaciones2)['Vehículo ID Registro'].count().reset_index()
df_grouped = df_grouped.merge(Matriculas_por_mma, how='left', on=['MMA']).fillna(0)
df_grouped['Rotación Mercado'] = (df_grouped['Vehículo ID Registro']/df_grouped['Matriculas'])*100
df_grouped
df_sorted = df_grouped.sort_values('Vehículo ID Registro', ascending=False)

df_sorted['porcentaje_ideal'] = df_sorted['Vehículo ID Registro']/sum(df_sorted['Vehículo ID Registro'])

df_sorted.rename(columns={'Vehículo ID Registro':'No Transacciones'}, inplace=True)


#Transacciones
df_sorted.head(3)

,MMA,No Transacciones,Matriculas,Rotación Mercado,porcentaje_ideal
233,CHEVROLET SAIL (2014-2016),326,23004.0,1.417145,0.019929
240,CHEVROLET SPARK (2014-2016),311,17184.0,1.809823,0.019012
239,CHEVROLET SPARK (2012-2013),268,18958.0,1.413651,0.016383


In [16]:
df_sorted.head(90)

,MMA,No Transacciones,Matriculas,Rotación Mercado,porcentaje_ideal
233,CHEVROLET SAIL (2014-2016),326,23004.0,1.417145,0.019929
240,CHEVROLET SPARK (2014-2016),311,17184.0,1.809823,0.019012
239,CHEVROLET SPARK (2012-2013),268,18958.0,1.413651,0.016383
186,CHEVROLET AVEO (2012-2013),261,20346.0,1.282807,0.015955
361,FORD FIESTA (2014-2016),240,14057.0,1.707334,0.014672
...,...,...,...,...,...
336,FORD ECOSPORT (2019-2020),52,0.0,inf,0.003179
220,CHEVROLET N300 (2012-2013),51,0.0,inf,0.003118
598,MAZDA 2 (2021-2023),51,1492.0,3.418231,0.003118
773,NISSAN FRONTIER (2014-2016),51,3036.0,1.679842,0.003118


#### Trends vs Stock

In [17]:
stockvsmercado = stock_sorted.merge(df_sorted, how='outer', on=['MMA']).fillna(0)
stockvsmercado = stockvsmercado.sort_values('No Transacciones', ascending=False)


# Regla de 3 D2S fuera de kavak
maximo_valor_A = np.sqrt(stockvsmercado['No Transacciones'].max())
D2Smin = 30
stockvsmercado['D2S Fuera Kavak'] = (D2Smin / np.sqrt(stockvsmercado['No Transacciones'])) * maximo_valor_A


stockvsmercado = stockvsmercado.replace([float('inf')], 1)

#merge con matriculas


stockvsmercado.loc[stockvsmercado['D2S Fuera Kavak'] < 2, 'D2S Fuera Kavak'] = 200
stockvsmercado['PC'] = stockvsmercado['AVG Inspections']/stockvsmercado['AVG Registers']
stockvsmercado.head(5)


,MMA,porcentaje,Stock por MMA,AVG MD,SD MD,aod_market_price,AVG D2S,AVG D2P,AVG Registers,AVG Inspections,...,Reservado L15D,Ventas L60D,AVG D2S Com,MD$,No Transacciones,Matriculas,Rotación Mercado,porcentaje_ideal,D2S Fuera Kavak,PC
46,CHEVROLET SAIL (2014-2016),0.006881,3.0,0.202387,0.017523,29735766.0,69.617875,28.254239,8.225806,0.0,...,2,0,41.363636,6.018135e+06,326.0,23004.0,1.417145,0.019929,30.000000,0.0
27,CHEVROLET SPARK (2014-2016),0.006881,3.0,0.191429,0.030943,24052986.0,63.261992,28.096047,4.619048,0.0,...,0,2,35.165945,4.604429e+06,311.0,17184.0,1.809823,0.019012,30.714953,0.0
172,CHEVROLET SPARK (2012-2013),0.002294,1.0,0.214545,0.044856,28586164.0,73.257177,38.529904,7.000000,0.0,...,0,0,34.727273,6.133032e+06,268.0,18958.0,1.413651,0.016383,33.087401,0.0
1416,CHEVROLET AVEO (2012-2013),0.000000,0.0,0.286000,0.016745,27508396.0,116.520970,38.520970,2.187500,0.0,...,0,0,78.000000,7.867401e+06,261.0,20346.0,1.282807,0.015955,33.528166,0.0
26,FORD FIESTA (2014-2016),0.006881,3.0,0.139909,0.014239,40006000.0,73.997247,28.019974,21.000000,0.0,...,1,0,45.977273,5.597203e+06,240.0,14057.0,1.707334,0.014672,34.964267,0.0


In [18]:
# stockvsmercado['Registros por SKU']

#### Acciones

In [19]:
#esto es para sacar el promedio de lso registros y luego pedirle a mkting que nos traiga mas registros de estos carros
filtered_data = [x for x in stockvsmercado['AVG Registers'] if x != 0]
mean_excluding_zeros = sum(filtered_data) / len(filtered_data)

In [20]:


# Define the conditions and corresponding values
conditions = [
    (stockvsmercado['Stock por MMA'] > 4), #Mas de 4 mma
    (stockvsmercado['Stock por MMA'] > 2) & (stockvsmercado['No Transacciones'] < 50), #Mas de 4 mma
    (stockvsmercado['Stock por MMA'] < 1) & (stockvsmercado['AVG MD'] > 0.25)  & (stockvsmercado['D2S Fuera Kavak'] < 110), # carros que tenemos bloqueados
    (stockvsmercado['aod_market_price'] < 30000000),  # A los de ticket <30.000.000 los deja quietos
    (stockvsmercado['Stock por MMA'] < 2) & (stockvsmercado['No Transacciones'] > 50) & (stockvsmercado['MD$'] > 7000000) & (stockvsmercado['PC'] < 0.05),  # Menos de un carro en Stock por MMA, No Transacciones > 50 y MD$ > 7000000
    (stockvsmercado['Stock por MMA'] > 1) & (stockvsmercado['No Transacciones'] < 50),  # Carros con Stock por MMA > 3 y No Transacciones < 50
    (stockvsmercado['AVG Registers'] < mean_excluding_zeros) & (stockvsmercado['D2S Fuera Kavak'] < 100) & (stockvsmercado['Stock por MMA'] == 0), # Si no tenemos registros
    (stockvsmercado['D2S Fuera Kavak'] < 60) & (stockvsmercado['Stock por MMA'] <2) # Si no tenemos registros

]

values = [
    'accion disponible (bloqueo)',
    'accion disponible (bloqueo)',
    'accion disponible (desbloqueo)',
    'En el minimo',
    'accion disponible (Premio)',
    'accion disponible (castigo)',
    'accion disponible (Sin registros, mkt)',
    'accion disponible (Sin registros, mkt)'
]

# Create a new column based on the conditions and values
stockvsmercado['accion'] = np.select(conditions, values, default='En el minimo')

# Define the conditions and corresponding values
conditions = [
    (stockvsmercado['Stock por MMA'] > 4), #Mas de 4 mma
    (stockvsmercado['Stock por MMA'] > 2) & (stockvsmercado['No Transacciones'] < 50), #Mas de 4 mma
    (stockvsmercado['Stock por MMA'] < 1) & (stockvsmercado['AVG MD'] > 0.25)  & (stockvsmercado['D2S Fuera Kavak'] < 110), # carros que tenemos bloqueados
    (stockvsmercado['aod_market_price'] < 30000000),  # A los de ticket <30.000.000 los deja quietos
    (stockvsmercado['Stock por MMA'] < 2) & (stockvsmercado['No Transacciones'] > 50) & (stockvsmercado['MD$'] > 7000000) & (stockvsmercado['PC'] < 0.05),  # Menos de un carro en Stock por MMA, No Transacciones > 50 y MD$ > 7000000
    (stockvsmercado['Stock por MMA'] > 1) & (stockvsmercado['No Transacciones'] < 50),  # Carros con Stock por MMA > 3 y No Transacciones < 50
    (stockvsmercado['AVG Registers'] < mean_excluding_zeros) & (stockvsmercado['D2S Fuera Kavak'] < 100) & (stockvsmercado['Stock por MMA'] == 0), # Si no tenemos registros
    (stockvsmercado['D2S Fuera Kavak'] < 60) & (stockvsmercado['Stock por MMA'] <2) # Si no tenemos registros

]

#Esto se le suma al MD por lo que debe ser negativo si se premia, positivo si se castiga
values = [
    0.15,
    0.15,
    -0.1,
    0,
    -0.02,
    0.02,
    0,
    0

]

# Create a new column based on the conditions and values
stockvsmercado['accion sobre MD'] = np.select(conditions, values, default=0)
#Reiniciar index
stockvsmercado = stockvsmercado.reset_index(drop=True)
stockvsmercado['Posición'] = stockvsmercado.index
stockvsmercado['AVG MD propuesto'] = stockvsmercado['AVG MD']+stockvsmercado['accion sobre MD']
stockvsmercado['AVG MD propuesto'] = stockvsmercado['AVG MD propuesto'].apply(lambda x: min(x, 0.30))
stockvsmercado['AVG MD propuesto'] = stockvsmercado['AVG MD propuesto'].apply(lambda x: max(x, 0.11))


In [21]:
stockvsmercado['PC'] = stockvsmercado['PC'].replace(np.inf, np.nan)
stockvsmercado.replace([np.inf, -np.inf], np.nan, inplace=True)
stockvsmercado = stockvsmercado.drop(columns=['PC'])


stockvsmercado.rename(columns={'Opps':'AVG Registers'}, inplace=True)


#### Carga google sheets

In [22]:
listoflist = stockvsmercado.values.tolist()
columns = stockvsmercado.columns.tolist()
listoflist = [columns] + stockvsmercado.values.tolist()


RANGE_NAME_UPDATE = 'Recomendaciones!A1'


request = sheet.values().update(spreadsheetId='xxxxxxxxx',
                                 range=RANGE_NAME_UPDATE,
                                 valueInputOption='USER_ENTERED',
                                 body={"values":listoflist})

response = request.execute()




print('DF cargado correctamente en Google Sheets.')

DF cargado correctamente en Google Sheets.
